In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import statsmodels.api as sm
import pingouin as pg
import sklearn
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("2023-08-31.csv", index_col = "Instrument")
df = df.dropna(axis=0)

In [3]:
def score_to_rank(array):
    res = np.argsort(np.flip(np.argsort(array)))+1
    return res

In [4]:
def par_corr(df): #get partial correlations. target variable must be the last col of df
    res = []
    for i in range(np.shape(df)[1]-1):
        col_names = list(df.columns[:-1])
        col_names.pop(i)
        par_corr = pg.partial_corr(df, df.columns[i], df.columns[-1], col_names)
        res.append(par_corr)
    return res

In [5]:
par_corr = par_corr(df)

In [6]:
df.head()

,Price Close,"Free Operating Cash Flow, Discrete",Total Debt,"PERCENT_CHG(TR.PriceClose(SDate=2023-08-31),lag=-10D)",Total Common Shares Outstanding,Cash and Equivalents,Net Income Reported - Actual,EBITDA - Mean,Interest Expense - Actual,Short Interest,"PERCENT_CHG(TR.Volume(SDate=2023-08-31),lag=-10D)",Current Assets,Noncurrent Asset,Current Liabilities,"ZAV(TR.PtoDPSMeanEst(SDate=2023-08-31,Period=FQ1))","ZAV(TR.PtoEPSMeanEst(SDate=2023-08-31,Period=FQ1))",Book Value Percentage of Market Capitalization (O/S),1 Month Total Return,Company Market Cap
Instrument,,,,,,,,,,,,,,,,,,,
FN.N,160.77,151039000.0,1.215600e+07,38.320571,36183680.0,401000.0,247913000.0,81550000.0,1472000.0,563589.0,128.180411,1.652540e+09,3.271080e+08,4.818850e+08,0.000,86.668464,31.153508,30.030734,5.818402e+09
TGLS.N,38.97,57724000.0,1.694840e+08,18.630137,47674770.0,103671000.0,155743000.0,76677170.0,8156000.0,888405.0,38.219532,4.321340e+08,3.021740e+08,2.098020e+08,487.125,40.113227,23.778543,-17.225998,1.857834e+09
ENV.N,54.63,-3288000.0,9.166550e+08,4.136485,54013830.0,162173000.0,-80939000.0,65498670.0,10897000.0,4031794.0,129.414857,3.052320e+08,1.806933e+09,3.270640e+08,0.000,101.435282,22.104582,-11.858664,2.979202e+09
PLOW.N,30.27,957000.0,2.064360e+08,-1.752678,22886790.0,20670000.0,38609000.0,24033330.0,11253000.0,175086.0,124.469821,2.529210e+08,3.439700e+08,1.004310e+08,0.000,63.951155,28.649817,-2.512077,6.957246e+08
GDOT.N,14.84,193360000.0,3.485433e+09,-5.657978,51674000.0,813945000.0,64212000.0,26755380.0,255000.0,1721117.0,110.977459,1.465832e+09,3.323344e+09,3.961671e+09,0.000,68.317834,93.968729,-24.092072,7.768218e+08


In [7]:
# get signs for monotonicity adjustment
signs = []
for i in range(len(par_corr)):
    signs.append(np.sign(par_corr[i]['r'][0]))

In [8]:
def normalize(array): # define function for normalization of scores
    maximum = np.max(array) 
    minimum = np.min(array)
    norm_list = []
    for i in range(len(array)):
        norm_list.append((array[i]-minimum)/(maximum-minimum))
    return np.array(norm_list)

In [9]:
# normalization of scores
for i in range(np.shape(df)[1]-1):
    df.iloc[:,i] = normalize(df.iloc[:,i]*signs[i])

In [10]:
# score combination by average weighting and feature importance weighting
def get_score_func(sys, no_feature, method):
    if method == 'average':
        feature_index = np.argsort(globals()['feature_importance_%s' % sys])[-no_feature:]
        score_func = np.zeros_like(df.iloc[:,0])
        for xi in feature_index:
            score_func += df.iloc[:,xi]
        score_func = score_func / no_feature
    elif method == 'importance':
        feature_index = np.argsort(globals()['feature_importance_%s' % sys])[-no_feature:]
        score_func = np.zeros_like(df.iloc[:,0])
        for xi in feature_index:
            score_func += df.iloc[:,xi]*globals()['feature_importance_%s' % sys][xi] / np.sum(np.sort(globals()['feature_importance_%s' % sys])[-no_feature:])
    return score_func

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
import itertools
import xgboost as xgb
import shap
shap.initjs

<function shap.plots._force.initjs()>

In [12]:
score_sys = ['lr', 'svm', 'rf', 'nn', 'xgb']
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [13]:
# conduct 80/20 random data splits for ten time
random_states = [123,234,345,456,567]
oos_r2_lr, oos_r2_svm, oos_r2_rf, oos_r2_nn, oos_r2_xgb = [], [], [], [], []
feature_importance_list_lr, feature_importance_list_svm, feature_importance_list_rf, feature_importance_list_nn, feature_importance_list_xgb = [], [], [], [], []
score_list_lr, score_list_svm, score_list_rf, score_list_nn, score_list_xgb = [], [], [], [], []
rank_list_lr, rank_list_svm, rank_list_rf, rank_list_nn, rank_list_xgb = [], [], [], [], []

for i in range(len(random_states)):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_states[i])
    
    # linear regression
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    oos_r2_lr.append(lr.score(X_test,y_test))
    #feature_importance_lr = permutation_importance(lr, X_train, y_train, random_state = random_states[i])['importances_mean']
    explainer = shap.Explainer(lr.predict, X_test)
    feature_importance_lr = np.mean(explainer(X_test).values, axis = 0)
    feature_importance_list_lr.append(feature_importance_lr)
    score_lr = get_score_func('lr', 8, 'importance')
    #score_lr = np.array(lr.predict(X))
    score_list_lr.append(score_lr)
    rank_list_lr.append(score_to_rank(score_lr))

    # svm
    estimator = SVR(kernel="rbf") # use rbf kernel when no. features smaller than no. of observations
    # fine tune sv regression and find best hyperparameters. defining parameter range
    param_grid = {'C': [0.1, 1, 10, 100, 1000], 'epsilon': [0.0001, 0.001, 0.01, 0.1, 0.5], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1]}
    svm = GridSearchCV(estimator, param_grid, cv = 5, verbose = 0, n_jobs = -1) # splitting data into 5-fold cv
    # fitting the model for grid search
    svm.fit(X_train, y_train)
    oos_r2_svm.append(svm.score(X_test,y_test))
    #feature_importance_svm = permutation_importance(svm, X_train, y_train, random_state = random_states[i])['importances_mean']
    explainer = shap.Explainer(svm.predict, X_test)
    feature_importance_svm = np.mean(explainer(X_test).values, axis = 0)
    feature_importance_list_svm.append(feature_importance_svm)
    score_svm = get_score_func('svm', 8, 'importance')
    #score_svm = np.array(svm.predict(X))
    score_list_svm.append(score_svm)
    rank_list_svm.append(score_to_rank(score_svm))

    # rf
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    oos_r2_rf.append(rf.score(X_test,y_test))
    #feature_importance_rf = permutation_importance(rf, X_train, y_train, random_state = random_states[i])['importances_mean']
    explainer = shap.Explainer(rf.predict, X_test)
    feature_importance_rf = np.mean(explainer(X_test).values, axis = 0)
    feature_importance_list_rf.append(feature_importance_rf)
    score_rf = get_score_func('rf', 8, 'importance')
    #score_rf = np.array(rf.predict(X))
    score_list_rf.append(score_rf)
    rank_list_rf.append(score_to_rank(score_rf))
    
    # nn
    mlp_regressor = MLPRegressor(random_state = random_states[i], max_iter=5000)
    mlp_regressor.fit(X_train, y_train)
    param_grid1 = {'activation': ['relu', 'tanh', 'logistic', 'identity'], 'hidden_layer_sizes': (12,12), 'solver': ['adam'], 'learning_rate' : ['constant', 'adaptive', 'invscaling']}
    nn = GridSearchCV(mlp_regressor, param_grid = param_grid1, n_jobs = -1, cv = 5, verbose = 0)
    nn.fit(X_train, y_train)
    oos_r2_nn.append(nn.score(X_test,y_test))
    #feature_importance_nn = permutation_importance(nn, X_train, y_train, random_state = random_states[i])['importances_mean']
    explainer = shap.Explainer(nn.predict, X_test)
    feature_importance_nn = np.mean(explainer(X_test).values, axis = 0)
    feature_importance_list_nn.append(feature_importance_nn)
    score_nn = get_score_func('nn', 8, 'importance')
    #score_nn = np.array(nn.predict(X))
    score_list_nn.append(score_nn)
    rank_list_nn.append(score_to_rank(score_nn))

    # xgb
    xgb_regressor = xgb.XGBRegressor(verbosity = 0)
    xgb_regressor.fit(X_train,y_train)
    param_grid2 = {'nthread':[4], 'objective':['reg:linear'], 'learning_rate': [.05, .10, .15], 'max_depth': [5, 6, 7], 'min_child_weight': [4], 'silent': [1], 'subsample': [0.8], 'colsample_bytree': [0.8], 'n_estimators': [500]}
    xgbr = GridSearchCV(xgb_regressor, param_grid2, cv = 5, n_jobs = -1, verbose = 0)
    xgbr.fit(X_train, y_train)
    oos_r2_xgb.append(xgbr.score(X_test,y_test))
    #feature_importance_xgb = permutation_importance(xgbr, X_train, y_train, random_state = random_states[i])['importances_mean']
    explainer = shap.Explainer(xgbr.predict, X_test)
    feature_importance_xgb = np.mean(explainer(X_test).values, axis = 0)
    feature_importance_list_xgb.append(feature_importance_xgb)
    score_xgb = get_score_func('xgb', 8, 'importance')
    #score_xgb = np.array(xgbr.predict(X))
    score_list_xgb.append(score_xgb)
    rank_list_xgb.append(score_to_rank(score_xgb))

Permutation explainer: 341it [38:23,  6.80s/it]                         
Permutation explainer: 341it [01:32,  3.26it/s]                         
/Users/NathanJiang/opt/anaconda3/envs/env-01/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/NathanJiang/opt/anaconda3/envs/env-01/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/NathanJiang/opt/anaconda3/envs/env-01/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/NathanJiang/opt/anaconda3/envs/env-01/lib/python3.11/si

In [14]:
# get diversity strength
ds_score = [[] for _ in range(len(random_states))]
ds_rank = [[] for _ in range(len(random_states))]

for sys in score_sys:
    for rs in range(len(ds_score)):
        loc = score_sys.index(sys)
        score_sys.remove(sys)
        ds = 0
        for i in range(len(score_sys)):
            ds += np.sum(np.square(normalize(np.sort(globals()['score_list_%s' % sys][rs]))-normalize(np.sort(globals()['score_list_%s' % score_sys[i]][rs]))))
        ds = ds / len(score_sys)
        score_sys.insert(loc, sys)
        ds_score[rs].append(ds)

ds_rank = np.reciprocal(ds_score)


In [15]:
# get performance strength
ps_score = [[] for _ in range(len(random_states))]

for sys in score_sys:
    for rs in range(len(ps_score)):
        ps = globals()['oos_r2_%s' % sys][rs]
        ps_score[rs].append(ps)

In [16]:
def powerset(s):
    x = len(s)
    ls = []
    for i in range(1 << x):
        ls.append([s[j] for j in range(x) if (i & (1 << j))])
    return ls[1:]

models = powerset(score_sys)

def myFunc(e):
  return len(e)

models.sort(key=myFunc)

models_list = []
for i in range(len(models)):
  if len(models[i]) == 1:
    models_list.append(models[i][0])
  elif len(models[i]) == 2:
    models_list.append(models[i][0]+'&'+models[i][1])
  elif len(models[i]) == 3:
    models_list.append(models[i][0]+'&'+models[i][1]+'&'+models[i][2])
  elif len(models[i]) == 4:
    models_list.append(models[i][0]+'&'+models[i][1]+'&'+models[i][2]+'&'+models[i][3])
  elif len(models[i]) == 5:
    models_list.append(models[i][0]+'&'+models[i][1]+'&'+models[i][2]+'&'+models[i][3]+'&'+models[i][4])

In [17]:
models_list

['lr',
 'svm',
 'rf',
 'nn',
 'xgb',
 'lr&svm',
 'lr&rf',
 'svm&rf',
 'lr&nn',
 'svm&nn',
 'rf&nn',
 'lr&xgb',
 'svm&xgb',
 'rf&xgb',
 'nn&xgb',
 'lr&svm&rf',
 'lr&svm&nn',
 'lr&rf&nn',
 'svm&rf&nn',
 'lr&svm&xgb',
 'lr&rf&xgb',
 'svm&rf&xgb',
 'lr&nn&xgb',
 'svm&nn&xgb',
 'rf&nn&xgb',
 'lr&svm&rf&nn',
 'lr&svm&rf&xgb',
 'lr&svm&nn&xgb',
 'lr&rf&nn&xgb',
 'svm&rf&nn&xgb',
 'lr&svm&rf&nn&xgb']

# Perform average score combinations

In [18]:
for rs in range(1, len(random_states)+1):
    globals()['avg_score_combine_rs%s' % rs] = pd.DataFrame({'lr':score_list_lr[rs-1], 'svm':score_list_svm[rs-1], 'rf':score_list_rf[rs-1], 'nn':score_list_nn[rs-1], 'xgb':score_list_xgb[rs-1]})

In [19]:
def avg_score_combine(models_list, single_score):
    for j in models_list[len(score_sys):]:
        if len(j.split('&')) == 2:
            single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]) / 2
        elif len(j.split('&')) == 3:
            single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]+single_score[j.split('&')[2]]) / 3
        elif len(j.split('&')) == 4:
            single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]+single_score[j.split('&')[2]]+single_score[j.split('&')[3]]) / 4
        elif len(j.split('&')) == 5:
            single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]+single_score[j.split('&')[2]]+single_score[j.split('&')[3]]+single_score[j.split('&')[4]]) / 5 

In [20]:
for rs in range(1, len(random_states)+1):
    avg_score_combine(models_list, globals()['avg_score_combine_rs%s' %rs])

# Perform average rank combinations

In [21]:
for rs in range(1, len(random_states)+1):
    globals()['avg_rank_combine_rs%s' % rs] = pd.DataFrame({'lr':score_to_rank(np.array(score_list_lr[rs-1])), 'svm':score_to_rank(np.array(score_list_svm[rs-1])), 'rf':score_to_rank(np.array(score_list_rf[rs-1])), 'nn':score_to_rank(np.array(score_list_nn[rs-1])), 'xgb':score_to_rank(np.array(score_list_xgb[rs-1]))}, index = avg_score_combine_rs1.index)

In [22]:
def avg_rank_combine(models_list, single_rank):
    for j in models_list[len(score_sys):]:
        if len(j.split('&')) == 2:
            single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]) / 2
        elif len(j.split('&')) == 3:
            single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]+single_rank[j.split('&')[2]]) / 3
        elif len(j.split('&')) == 4:
            single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]+single_rank[j.split('&')[2]]+single_rank[j.split('&')[3]]) / 4
        elif len(j.split('&')) == 5:
            single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]+single_rank[j.split('&')[2]]+single_rank[j.split('&')[3]]+single_rank[j.split('&')[4]]) / 5

In [23]:
for rs in range(1, len(random_states)+1):
    avg_rank_combine(models_list, globals()['avg_rank_combine_rs%s' %rs])

# Perform weighted score combination by diversity strength

In [24]:
for rs in range(1, len(random_states)+1):
    globals()['ds_score_combine_rs%s' % rs] = pd.DataFrame()

In [25]:
def ds_score_combine(models_list, single_score, ds_score_combine, ds_score):
    for j in models_list[len(score_sys):]:
        if len(j.split('&')) == 2:
            ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[score_sys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[score_sys.index(j.split('&')[1])])/(ds_score[score_sys.index(j.split('&')[0])] + ds_score[score_sys.index(j.split('&')[1])])
        elif len(j.split('&')) == 3:
            ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[score_sys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[score_sys.index(j.split('&')[1])]+single_score[j.split('&')[2]]*ds_score[score_sys.index(j.split('&')[2])])/(ds_score[score_sys.index(j.split('&')[0])] + ds_score[score_sys.index(j.split('&')[1])] + ds_score[score_sys.index(j.split('&')[2])])
        elif len(j.split('&')) == 4:
            ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[score_sys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[score_sys.index(j.split('&')[1])]+single_score[j.split('&')[2]]*ds_score[score_sys.index(j.split('&')[2])]+single_score[j.split('&')[3]]*ds_score[score_sys.index(j.split('&')[3])])/(ds_score[score_sys.index(j.split('&')[0])] + ds_score[score_sys.index(j.split('&')[1])] + ds_score[score_sys.index(j.split('&')[2])] + ds_score[score_sys.index(j.split('&')[3])])
        elif len(j.split('&')) == 5:
            ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[score_sys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[score_sys.index(j.split('&')[1])]+single_score[j.split('&')[2]]*ds_score[score_sys.index(j.split('&')[2])]+single_score[j.split('&')[3]]*ds_score[score_sys.index(j.split('&')[3])]+single_score[j.split('&')[4]]*ds_score[score_sys.index(j.split('&')[4])])/(ds_score[score_sys.index(j.split('&')[0])] + ds_score[score_sys.index(j.split('&')[1])] + ds_score[score_sys.index(j.split('&')[2])] + ds_score[score_sys.index(j.split('&')[3])] + ds_score[score_sys.index(j.split('&')[4])])

In [26]:
for rs in range(1, len(random_states)+1):
    ds_score_combine(models_list, globals()['avg_score_combine_rs%s' %rs], globals()['ds_score_combine_rs%s' %rs], ds_score[rs-1])

# Perform weighted rank combination by diversity strength

In [27]:
for rs in range(1, len(random_states)+1):
    globals()['ds_rank_combine_rs%s' % rs] = pd.DataFrame()

In [28]:
def ds_rank_combine(models_list, single_rank, ds_rank_combine, ds_rank):
    for j in models_list[len(score_sys):]:
        if len(j.split('&')) == 2:
            ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[score_sys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[score_sys.index(j.split('&')[1])])/(ds_rank[score_sys.index(j.split('&')[0])] + ds_rank[score_sys.index(j.split('&')[1])])
        elif len(j.split('&')) == 3:
            ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[score_sys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[score_sys.index(j.split('&')[1])]+single_rank[j.split('&')[2]]*ds_rank[score_sys.index(j.split('&')[2])])/(ds_rank[score_sys.index(j.split('&')[0])] + ds_rank[score_sys.index(j.split('&')[1])] + ds_rank[score_sys.index(j.split('&')[2])])
        elif len(j.split('&')) == 4:
            ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[score_sys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[score_sys.index(j.split('&')[1])]+single_rank[j.split('&')[2]]*ds_rank[score_sys.index(j.split('&')[2])]+single_rank[j.split('&')[3]]*ds_rank[score_sys.index(j.split('&')[3])])/(ds_rank[score_sys.index(j.split('&')[0])] + ds_rank[score_sys.index(j.split('&')[1])] + ds_rank[score_sys.index(j.split('&')[2])] + ds_rank[score_sys.index(j.split('&')[3])])
        elif len(j.split('&')) == 5:
            ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[score_sys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[score_sys.index(j.split('&')[1])]+single_rank[j.split('&')[2]]*ds_rank[score_sys.index(j.split('&')[2])]+single_rank[j.split('&')[3]]*ds_rank[score_sys.index(j.split('&')[3])]+single_rank[j.split('&')[4]]*ds_rank[score_sys.index(j.split('&')[4])])/(ds_rank[score_sys.index(j.split('&')[0])] + ds_rank[score_sys.index(j.split('&')[1])] + ds_rank[score_sys.index(j.split('&')[2])] + ds_rank[score_sys.index(j.split('&')[3])] + ds_rank[score_sys.index(j.split('&')[4])])

In [29]:
for rs in range(1, len(random_states)+1):
    ds_rank_combine(models_list, globals()['avg_rank_combine_rs%s' %rs], globals()['ds_rank_combine_rs%s' %rs], ds_rank[rs-1])

# Perform weighted score combination by performance strength

In [30]:
for rs in range(1, len(random_states)+1):
    globals()['ps_score_combine_rs%s' % rs] = pd.DataFrame()

In [31]:
def ps_score_combine(models_list, single_score, ps_score_combine, ps_score):
    for j in models_list[len(score_sys):]:
        if len(j.split('&')) == 2:
            ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[score_sys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[score_sys.index(j.split('&')[1])]))/(ps_score[score_sys.index(j.split('&')[0])] + ps_score[score_sys.index(j.split('&')[1])])
        elif len(j.split('&')) == 3:
            ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[score_sys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[score_sys.index(j.split('&')[1])])+single_score[j.split('&')[2]]*(ps_score[score_sys.index(j.split('&')[2])]))/(ps_score[score_sys.index(j.split('&')[0])] + ps_score[score_sys.index(j.split('&')[1])] + ps_score[score_sys.index(j.split('&')[2])])
        elif len(j.split('&')) == 4:
            ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[score_sys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[score_sys.index(j.split('&')[1])])+single_score[j.split('&')[2]]*(ps_score[score_sys.index(j.split('&')[2])])+single_score[j.split('&')[3]]*(ps_score[score_sys.index(j.split('&')[3])]))/(ps_score[score_sys.index(j.split('&')[0])] + ps_score[score_sys.index(j.split('&')[1])] + ps_score[score_sys.index(j.split('&')[2])] + ps_score[score_sys.index(j.split('&')[3])])
        elif len(j.split('&')) == 5:
            ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[score_sys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[score_sys.index(j.split('&')[1])])+single_score[j.split('&')[2]]*(ps_score[score_sys.index(j.split('&')[2])])+single_score[j.split('&')[3]]*(ps_score[score_sys.index(j.split('&')[3])])+single_score[j.split('&')[4]]*(ps_score[score_sys.index(j.split('&')[4])]))/(ps_score[score_sys.index(j.split('&')[0])] + ps_score[score_sys.index(j.split('&')[1])] + ps_score[score_sys.index(j.split('&')[2])] + ps_score[score_sys.index(j.split('&')[3])] + ps_score[score_sys.index(j.split('&')[4])])

In [32]:
for rs in range(1, len(random_states)+1):
    ps_score_combine(models_list, globals()['avg_score_combine_rs%s' %rs], globals()['ps_score_combine_rs%s' %rs], ps_score[rs-1])

# Perform weighted rank combination by performance strength

In [33]:
for rs in range(1, len(random_states)+1):
    globals()['ps_rank_combine_rs%s' % rs] = pd.DataFrame()

In [34]:
def ps_rank_combine(models_list, single_rank, ps_rank_combine, ps_score):
    for j in models_list[len(score_sys):]:
        if len(j.split('&')) == 2:
            ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1/ps_score[score_sys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1/ps_score[score_sys.index(j.split('&')[1])]))/(1/ps_score[score_sys.index(j.split('&')[0])] + 1/ps_score[score_sys.index(j.split('&')[1])])
        elif len(j.split('&')) == 3:
            ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1/ps_score[score_sys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1/ps_score[score_sys.index(j.split('&')[1])])+single_rank[j.split('&')[2]]*(1/ps_score[score_sys.index(j.split('&')[2])]))/(1/ps_score[score_sys.index(j.split('&')[0])] + 1/ps_score[score_sys.index(j.split('&')[1])] + 1/ps_score[score_sys.index(j.split('&')[2])])
        elif len(j.split('&')) == 4:
            ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1/ps_score[score_sys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1/ps_score[score_sys.index(j.split('&')[1])])+single_rank[j.split('&')[2]]*(1/ps_score[score_sys.index(j.split('&')[2])])+single_rank[j.split('&')[3]]*(1/ps_score[score_sys.index(j.split('&')[3])]))/(1/ps_score[score_sys.index(j.split('&')[0])] + 1/ps_score[score_sys.index(j.split('&')[1])] + 1/ps_score[score_sys.index(j.split('&')[2])] + 1/ps_score[score_sys.index(j.split('&')[3])])
        elif len(j.split('&')) == 5:
            ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1/ps_score[score_sys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1/ps_score[score_sys.index(j.split('&')[1])])+single_rank[j.split('&')[2]]*(1/ps_score[score_sys.index(j.split('&')[2])])+single_rank[j.split('&')[3]]*(1/ps_score[score_sys.index(j.split('&')[3])])+single_rank[j.split('&')[4]]*(1/ps_score[score_sys.index(j.split('&')[4])]))/(1/ps_score[score_sys.index(j.split('&')[0])] + 1/ps_score[score_sys.index(j.split('&')[1])] + 1/ps_score[score_sys.index(j.split('&')[2])] + 1/ps_score[score_sys.index(j.split('&')[3])] + 1/ps_score[score_sys.index(j.split('&')[4])])

In [35]:
for rs in range(1, len(random_states)+1):
    ps_rank_combine(models_list, globals()['avg_rank_combine_rs%s' %rs], globals()['ps_rank_combine_rs%s' %rs], ps_score[rs-1])

In [36]:
for rs in range(1, len(random_states)+1):
    globals()['avg_rank_combine_rs%s' %rs].rename(columns={'lr': 'lr_r', 'svm': 'svm_r', 'rf': 'rf_r', 'nn': 'nn_r', 'xgb': 'xgb_r'}, inplace=True)

In [37]:
def spearman_corr(pred_rank, actual_rank):
    n = len(pred_rank)
    res = 1 - 6 * np.sum((pred_rank - actual_rank)**2) / (n*(n**2-1))
    return res

In [38]:
actual_rank = score_to_rank(np.array(df.iloc[:,-1]))

In [39]:
for i in range(1, len(random_states)+1):
    globals()['pred_rank_rs%s' %i] = pd.concat([globals()['avg_score_combine_rs%s' %i], globals()['ds_score_combine_rs%s' %i], globals()['ps_score_combine_rs%s' %i], globals()['avg_rank_combine_rs%s' %i], globals()['ds_rank_combine_rs%s' %i], globals()['ps_rank_combine_rs%s' %i]], axis = 1)

In [40]:
for i in range(1, len(random_states)+1):
    n = len(score_sys)
    m = 3 # number of weighting schemes
    for j in range(n+(2**n-n-1)*m):
        globals()['pred_rank_rs%s' %i].iloc[:,j] = score_to_rank(np.array(globals()['pred_rank_rs%s' %i].iloc[:,j]))
    for j in range((n+(2**n-n-1)*m), ((n+(2**n-n-1)*m)*2)):
        globals()['pred_rank_rs%s' %i].iloc[:,j] = score_to_rank(np.array(-globals()['pred_rank_rs%s' %i].iloc[:,j]))

In [41]:
# get one month total returns for stocks ranked top 100 by each combination

one_month_returns = np.array(df.iloc[:,-1])

for rs in range(1, len(random_states)+1):
    globals()['trun_returns_rs%s' % rs] = pd.DataFrame()

for rs in range(1, len(random_states)+1):
    for j in globals()['pred_rank_rs%s' %rs].columns:
        globals()['trun_returns_rs%s' % rs][j] = one_month_returns[np.argsort(np.array(globals()['pred_rank_rs%s' %rs][j]))[:100]]

In [42]:
for rs in range(1, len(random_states)+1):
    globals()['trun_returns_rs%s' %rs].to_csv('top_returns_20230831_rs'+str(rs)+'.csv')

In [43]:
final_ranking = (pred_rank_rs1['lr&rf&xgb'] + pred_rank_rs2['lr&svm&xgb_ps_r'] + pred_rank_rs3['lr'] + pred_rank_rs4['lr&rf&nn&xgb'] + pred_rank_rs5['svm&nn&xgb_ps']) / len(random_states)
final_ranking = score_to_rank(-np.array(final_ranking))
final_ranking = pd.DataFrame({'final_ranking': final_ranking}, index = df.index)

In [44]:
final_ranking.sort_values(by = 'final_ranking').to_csv('final_ranking_for_Sep2023.csv')